# Definition of a memory index

31/10/2020

1. We compute the mean expression of the ON-memory genes from bulk RNA-seq data in each cell

2. Divide by the maximum over all the cells

3. Violin plot of the memory index per cluster divided by condition

4. Test clusters for significant memory


Note: this analysis should be revised

In [2]:
%matplotlib inline

In [ ]:
library("Seurat")

xenopus.combined <- readRDS(file = "xenopus_combined_and_clustered_no_out.rds")

In [ ]:
import pandas as pd

ON_mem_df=pd.read_csv('EndoNT_Filter_DonorsvsIVFdownFDR_DonsRPKM1_IVFvsNT_FDRup__OnMemFDR.csv')

ON_mem_old=list(ON_mem_df['Row.names'])

ON_mem_new=[]

for i in ON_mem_old:
    lst=i.split('_')
    if 'unnamed' in lst[1]:
        ON_mem_new.append(lst[0])
    else:
        ON_mem_new.append(lst[1])
len(ON_mem_new)

In [ ]:
%get ON_mem_new

xenopus.subset <- subset(xenopus.combined,idents=c('0','1','2','3','4','5','6','7','8','9'))
xenopus.subset <- subset(xenopus.subset, features=ON_mem_new)
mat.xenopus <- as.data.frame(GetAssayData(object = xenopus.subset))
metadata.xenopus <- xenopus.subset[[]]

In [ ]:
%get mat.xenopus --from R
%get metadata.xenopus --from R

mat_xenopus=mat_xenopus.T
mat_xenopus_mean=pd.DataFrame(mat_xenopus.mean(axis=1))
#mat_xenopus_mean/=mat_xenopus_mean.max()

In [ ]:
from scipy.stats import mannwhitneyu

mat_xenopus_mean['isnt']=metadata_xenopus['isnt']
mat_xenopus_mean['seurat_clusters']=metadata_xenopus['seurat_clusters']

# Perform Wilcoxon test between IVF and NT in each cluster
for i in range(10):
    tmp_df=mat_xenopus_mean[mat_xenopus_mean['seurat_clusters']==str(i)]
    tmp_df_IVF=tmp_df[tmp_df['isnt']=='IVF']
    tmp_df_NT=tmp_df[tmp_df['isnt']=='NT']
    print(i,mannwhitneyu(tmp_df_IVF[0],tmp_df_NT[0])[1])
    

In [ ]:
%get mat_xenopus_mean

xenopus.subset[['memory.index']] <- mat_xenopus_mean

VlnPlot(xenopus.subset, features = 'memory.index' ,split.by='stim')
VlnPlot(xenopus.subset, features = 'memory.index' ,split.by='isnt')

FeaturePlot(xenopus.subset,pt.size=3, reduction = "umap", features = "memory.index")

# Entropy of batch mixing

Look at [Haghverdi et al, 2018](https://www.nature.com/articles/nbt.4091), section "Entropy of batch mixing"

TO BE DONE